**Part 1: Harris Corner Detector**

In [1]:
# importing libraries

import cv2 
import sys

import numpy as np
import matplotlib as plt

np.set_printoptions(threshold=sys.maxsize)

In [2]:
# loading example image

img = cv2.imread('CV_Lista02_dados/goi1.jpg', 0)

In [3]:
# Calculating derivatives for both directions
Wd = 3

dx = cv2.Sobel(img, -1, 1, 0, ksize=Wd)
dy = cv2.Sobel(img, -1, 0, 1, ksize=Wd)

In [4]:
# Visualizing dx

cv2.imshow('img', dx)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# Visualizing dy

cv2.imshow('img', dy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# Setting relations for the covariance matrix

dxdx = dx ** 2
dydy = dy ** 2
dxdy = dx * dy

In [7]:
# Initializing parameters
width = img.shape[0]
heigh = img.shape[1]
offset = int(((5 * Wd) - 1) / 2)
threshold = 1500000
stride = 1

# Initializing corner variable to store the points
corner = np.zeros((width, heigh), np.float64)

# Applying symmetric padding to the image
np.pad(dxdx, ((offset, offset), (offset, offset)), 'constant')
np.pad(dydy, ((offset, offset), (offset, offset)), 'constant')
np.pad(dxdy, ((offset, offset), (offset, offset)), 'constant')

# For each pixel, the covariance matrix is calculated
for y in range(heigh):
    for x in range(width):
        sdxdx = np.sum(dxdx[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])
        sdydy = np.sum(dydy[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])
        sdxdy = np.sum(dxdy[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])
        
        cov = [[sdxdx, sdxdy], [sdxdy, sdydy]]
        
        lamb1 = np.linalg.eig(cov)[0][0]
        lamb2 = np.linalg.eig(cov)[0][1]
        
        R = ((lamb1 * lamb2) - 0.04 * ((lamb1 + lamb2) ** 2))
        
        if R > threshold:
            corner[x,y] += R

In [8]:
# Visualizing the points detected as corners

cv2.imshow('img', corner)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Setting a window Wh to verify which corners are to be considered

Wh = 15

idxs = []

filt_corner = np.zeros((width, heigh, 1), np.float64)

for y in range (heigh // Wh):
    for x in range(width // Wh):
        idx = np.argmax(corner[(x * Wh) : ((x + 1) * Wh), (y * Wh) : ((y + 1) * Wh)])
#         print('x: ', x)
#         print('y: ', y)
#         print('idx: ', idx)
        
        if (idx != 0):
            y2 = ((y * Wh) + (idx % Wh))
            x2 = ((x * Wh) + (idx // Wh))
            idxs.append(idx)
            filt_corner[x2,y2] = 255

In [10]:
# Visualizing

cv2.imshow('img', filt_corner)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
kernel = np.ones((Wd, Wd), np.uint8)
fc_dil = cv2.dilate(filt_corner, kernel, iterations=1)

In [12]:
# Visualizing

cv2.imshow('img', fc_dil)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# res = cv2.bitwise_and(img, img, mask=fc_dil)

In [15]:
# # Visualizing

# cv2.imshow('img', res)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

**Part 2: Feature Crossing**